In [1]:
## Need to add collapse across wins/loss for partner to the heatmap
## Use absolute value for that ^^
## make sure to demean all the current output
## add dummy code for study ID to capture third flip angle

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [3]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

In [4]:
script_dir = os.getcwd()

In [5]:
bids_dir = os.path.join(os.getcwd(), '..', 'ds003745')
bids_dir = os.path.abspath(bids_dir)

In [6]:
age_srndna_path = os.path.join(bids_dir, 'participants.tsv')
age_rf1_path = os.path.join(bids_dir, 'participants-rf1.tsv')

In [7]:
age_srndna = pd.read_table(age_srndna_path)
age_rf1 = pd.read_table(age_rf1_path)
age_df = pd.concat([age_srndna, age_rf1])
age_df = age_df.rename(columns={"participant_id": "sub"})
age_df['sub'] = age_df['sub'].str.replace('^sub-', '', regex=True)
age_df['sub'] = age_df['sub'].astype(int)
display(age_df)

,sub,age,sex,group
0,106,20,F,control
1,109,28,F,control
2,104,20,M,control
3,110,20,M,control
4,105,21,F,control
...,...,...,...,...
130,10964,60,F,control
131,11068,43,F,control
132,11071,62,M,control
133,11072,25,M,control


In [8]:
logs = os.path.join(script_dir, 'combined_logs_with_age.csv')
logs_df = pd.read_csv(logs)
display(logs_df)

,sub,trial,partner,trait,ran,order,response,age
0,103,1,3,0,1,0,3,NaN
1,103,2,2,0,1,1,1,NaN
2,103,3,1,0,1,2,1,NaN
3,103,4,3,1,1,3,1,NaN
4,103,5,2,1,1,4,0,NaN
...,...,...,...,...,...,...,...,...
1021,11072,2,2,0,1,1,0,25.0
1022,11072,3,1,0,1,2,0,25.0
1023,11072,4,3,1,1,3,0,25.0
1024,11072,5,2,1,1,4,0,25.0


In [9]:
fmri_metrics = os.path.join(script_dir, 'exclusions_tsnr-fdmean.csv') # This df stores tsnr and fdmean 
metrics_df = pd.read_csv(fmri_metrics)
display(metrics_df)

,Subject,Run,vsmean_stan,fd_mean
0,104,1,91.703647,0.081289
1,104,2,94.158314,0.078489
2,105,1,107.323792,0.084580
3,105,2,106.397772,0.078785
4,106,1,106.132565,0.161061
...,...,...,...,...
279,10983,2,57.124313,0.262444
280,10984,1,40.239686,0.232690
281,10984,2,38.185517,0.181121
282,11031,1,56.074851,0.129847


In [10]:
fsl_inputs = os.path.join(script_dir, 'L3inputs.csv') # This df stores the relevant FSL inputs
fsl_df = pd.read_csv(fsl_inputs)
print(fsl_df)

                                                  path
0    /ZPOOL/data/projects/sharedreward-aging/deriva...
1    /ZPOOL/data/projects/sharedreward-aging/deriva...
2    /ZPOOL/data/projects/sharedreward-aging/deriva...
3    /ZPOOL/data/projects/sharedreward-aging/deriva...
4    /ZPOOL/data/projects/sharedreward-aging/deriva...
..                                                 ...
137  /ZPOOL/data/projects/sharedreward-aging/deriva...
138  /ZPOOL/data/projects/sharedreward-aging/deriva...
139  /ZPOOL/data/projects/sharedreward-aging/deriva...
140  /ZPOOL/data/projects/sharedreward-aging/deriva...
141  /ZPOOL/data/projects/sharedreward-aging/deriva...

[142 rows x 1 columns]


In [11]:
flip_angle =  os.path.join(script_dir, 'flip.csv') # This df stores the flip angle designation to be used for dummy variable coding
flip_df = pd.read_csv(flip_angle)
display(flip_df)

,sub,flip
0,10391,0
1,10460,0
2,10555,0
3,10369,0
4,10596,0
...,...,...
160,10977,1
161,10983,1
162,10984,1
163,11005,1


In [12]:
# This cell is used to clean the logs files so that each sub only has one row
# Need to add code to drop index column

# Pivot the dataframe to reshape it so that each subject has a single row
pivot_df = logs_df.pivot_table(index='sub', columns=['partner', 'trait'], values='response')

# Flatten the MultiIndex columns
pivot_df.columns = [f'partner_{col[0]}_trait_{col[1]}' for col in pivot_df.columns]

# Merge the age information back
pivot_df = pivot_df.merge(logs_df[['sub', 'age']].drop_duplicates(), on='sub', how='left')

# Reset index to make 'sub' a column again
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df.drop(['index'], axis = 1)


# Print the shape of the reformatted dataframe
#print("Shape of the reformatted dataframe:", pivot_df.shape)
display(pivot_df.head())

,sub,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,age
0,103,1.0,0.0,1.0,0.0,3.0,1.0,NaN
1,104,5.0,-1.0,5.0,0.0,5.0,0.0,20.0
2,107,0.0,0.0,5.0,0.0,5.0,-2.0,23.0
3,108,1.0,0.0,4.0,-3.0,5.0,-5.0,21.0
4,111,5.0,0.0,5.0,5.0,5.0,5.0,80.0


In [13]:
# This cell is responsbile for adding the correct L3 input to the subject row
fsl_df['Subject'] = fsl_df['path'].str.extract(r'sub-(\d+)').astype(int)
fsl_df['Run'] = fsl_df['path'].str.extract(r'run-(\d+)').fillna('3').astype(int)
display(fsl_df)

,path,Subject,Run
0,/ZPOOL/data/projects/sharedreward-aging/deriva...,104,3
1,/ZPOOL/data/projects/sharedreward-aging/deriva...,105,3
2,/ZPOOL/data/projects/sharedreward-aging/deriva...,106,3
3,/ZPOOL/data/projects/sharedreward-aging/deriva...,107,3
4,/ZPOOL/data/projects/sharedreward-aging/deriva...,108,3
...,...,...,...
137,/ZPOOL/data/projects/sharedreward-aging/deriva...,10974,2
138,/ZPOOL/data/projects/sharedreward-aging/deriva...,10977,3
139,/ZPOOL/data/projects/sharedreward-aging/deriva...,10983,3
140,/ZPOOL/data/projects/sharedreward-aging/deriva...,10984,3


In [14]:
# Now that we have our list of inputs, we can pull out the proper tsnr and fdmean for each subject
# Metric values should be derived from the same L1 run as the L3 input, if L2, use the average of both runs
# We are also going to do the math in this cell for averaging the variables

# Pivot the DataFrame to reshape it
parsed_metrics_df = metrics_df.pivot(index='Subject', columns='Run')

# Flatten the MultiIndex columns
parsed_metrics_df.columns = [f'{var}_{run}' for var, run in parsed_metrics_df.columns]

# Create the averages and the columns for them
parsed_metrics_df['avg_tsnr'] = parsed_metrics_df[['vsmean_stan_1', 'vsmean_stan_2']].mean(axis=1)
parsed_metrics_df['avg_fdmean'] = parsed_metrics_df[['fd_mean_1', 'fd_mean_2']].mean(axis=1)

# Print the reshaped DataFrame
display(parsed_metrics_df)

,vsmean_stan_1,vsmean_stan_2,fd_mean_1,fd_mean_2,avg_tsnr,avg_fdmean
Subject,,,,,,
104,91.703647,94.158314,0.081289,0.078489,92.930981,0.079889
105,107.323792,106.397772,0.084580,0.078785,106.860782,0.081682
106,106.132565,95.061572,0.161061,0.165864,100.597069,0.163462
107,106.384215,108.987225,0.078820,0.087712,107.685720,0.083266
108,80.922331,77.318246,0.123056,0.163046,79.120289,0.143051
...,...,...,...,...,...,...
10974,NaN,47.765127,NaN,0.292531,47.765127,0.292531
10977,43.972848,46.337484,0.089736,0.094241,45.155166,0.091988
10983,55.970331,57.124313,0.296016,0.262444,56.547322,0.279230


In [15]:
# Merge the metrics df with the L3inputs (fsl) df and drop unecessary columns
metric_run_merge_df = pd.merge(fsl_df, parsed_metrics_df, on='Subject')
metric_run_merge_df = pd.merge(fsl_df, parsed_metrics_df, on='Subject')


# Create the new columns based on 'Run'
metric_run_merge_df['tsnr'] = metric_run_merge_df.apply(
    lambda row: row['vsmean_stan_1'] if row['Run'] == 1 else (row['vsmean_stan_2'] if row['Run'] == 2 else row['avg_tsnr']),
    axis=1
)
metric_run_merge_df['fdmean'] = metric_run_merge_df.apply(
    lambda row: row['fd_mean_1'] if row['Run'] == 1 else (row['fd_mean_2'] if row['Run'] == 2 else row['avg_fdmean']),
    axis=1
)


# Add dummy coding for study ID (SRNDNA=0 and RF1=1)
metric_run_merge_df['study'] = metric_run_merge_df['Subject'].apply(lambda x: 0 if len(str(x)) == 3 else 1)

# Add columns of ones
metric_run_merge_df['ones'] = metric_run_merge_df['Subject'].apply(lambda x: '1')


# Rename Subject column to sub for reusability
metric_run_merge_df = metric_run_merge_df.rename(columns={"Subject": "sub"})
metric_run_merge_df = pd.merge(flip_df, metric_run_merge_df, on='sub')
metric_run_merge_df = pd.merge(age_df, metric_run_merge_df, on='sub')
metric_run_merge_df = metric_run_merge_df[['sub', 'age', 'tsnr', 'fdmean', 'flip', 'study']]


# Print the new DataFrame
print(metric_run_merge_df)

       sub  age        tsnr    fdmean  flip  study
0      106   20  100.597069  0.163462     0      0
1      104   20   92.930981  0.079889     0      0
2      110   20   86.559506  0.089279     0      0
3      105   21  106.860782  0.081682     0      0
4      108   21   79.120289  0.143051     0      0
..     ...  ...         ...       ...   ...    ...
132  10983   24   56.547322  0.279230     1      1
133  10974   80   47.765127  0.292531     1      1
134  11031   23   55.706430  0.123523     1      1
135  10956   28   54.431564  0.086321     1      1
136  10984   52   39.212601  0.206906     1      1

[137 rows x 6 columns]


In [25]:
import pandas as pd

# Define columns to demean, excluding 'sub', 'flip', and 'study'
columns_to_demean = metric_run_merge_df.columns.difference(['sub', 'flip', 'study'])

# Demean the selected columns
demeaned_df = metric_run_merge_df[columns_to_demean].apply(lambda x: x - x.mean(), axis=0)

# Combine the demeaned columns with the original 'B' column
result_df = pd.concat([demeaned_df, metric_run_merge_df[['sub', 'flip', 'study']]], axis=1)

# Add 'ones' column as '1' based on 'sub'
result_df['ones'] = result_df['sub'].apply(lambda x: '1')

# Reorder columns with 'sub' and 'ones' at the beginning
columns = result_df.columns.tolist()
columns.insert(0, columns.pop(columns.index('sub')))
columns.insert(1, columns.pop(columns.index('ones')))
result_df = result_df[columns]

# Display the result
display(result_df)

# Save to CSV
result_df.to_csv('result_df.csv', index=False)


,sub,ones,age,fdmean,tsnr,flip,study
0,106,1,-22.284672,-0.013875,50.258886,0,0
1,104,1,-22.284672,-0.097448,42.592798,0,0
2,110,1,-22.284672,-0.088058,36.221324,0,0
3,105,1,-21.284672,-0.095655,56.522599,0,0
4,108,1,-21.284672,-0.034286,28.782106,0,0
...,...,...,...,...,...,...,...
132,10983,1,-18.284672,0.101893,6.209139,1,1
133,10974,1,37.715328,0.115194,-2.573056,1,1
134,11031,1,-19.284672,-0.053814,5.368247,1,1
135,10956,1,-14.284672,-0.091017,4.093381,1,1


In [31]:
## Now it will make the most sense to merge the inputs to the covariates dataframe for quality assurance and easier copy pasting

# Re-extract the sub variable from the L3inputs
fsl_df['sub'] = fsl_df['path'].str.extract(r'sub-(\d+)').astype(int)
# fsl_df = fsl_df.drop(['Subject', 'Run'], axis = 1)
columns = fsl_df.columns.tolist()
columns.insert(0, columns.pop(columns.index('sub')))
fsl_df = fsl_df[columns]

# Merge result
final_df = pd.merge(result_df, fsl_df, on='sub')
final_df = final_df.drop(['Subject', 'Run'], axis = 1)
display(final_df)
final_df.to_csv('final_df.csv', index=False)


,sub,ones,age,fdmean,tsnr,flip,study,path
0,106,1,-22.284672,-0.013875,50.258886,0,0,/ZPOOL/data/projects/sharedreward-aging/deriva...
1,104,1,-22.284672,-0.097448,42.592798,0,0,/ZPOOL/data/projects/sharedreward-aging/deriva...
2,110,1,-22.284672,-0.088058,36.221324,0,0,/ZPOOL/data/projects/sharedreward-aging/deriva...
3,105,1,-21.284672,-0.095655,56.522599,0,0,/ZPOOL/data/projects/sharedreward-aging/deriva...
4,108,1,-21.284672,-0.034286,28.782106,0,0,/ZPOOL/data/projects/sharedreward-aging/deriva...
...,...,...,...,...,...,...,...,...
132,10983,1,-18.284672,0.101893,6.209139,1,1,/ZPOOL/data/projects/sharedreward-aging/deriva...
133,10974,1,37.715328,0.115194,-2.573056,1,1,/ZPOOL/data/projects/sharedreward-aging/deriva...
134,11031,1,-19.284672,-0.053814,5.368247,1,1,/ZPOOL/data/projects/sharedreward-aging/deriva...
135,10956,1,-14.284672,-0.091017,4.093381,1,1,/ZPOOL/data/projects/sharedreward-aging/deriva...


In [18]:
# Table for basic model
#table_sub = full_covs_df[['sub', 'age',
                      'tsnr', 'fdmean', 'flip']]

#display(table_sub)

IndentationError: unexpected indent (2255999179.py, line 3)

In [ ]:
# This cell pulls ratings and creates the difference columns. 
# FYI: Partner vals: 3=Friend, 2=Stranger, 1=Computer
# FYI: Trait vals: 0=How did it feel to win, 1=How did it feel to lose

# Load the reformatted CSV file
data = pivot_df
# List of partner combinations
partner_pairs = [(3, 2), (3, 1)]

# Create a new DataFrame to store the differences
diff_df = pd.DataFrame()

# Iterate over each trait
for trait in [0, 1]:
    for (p1, p2) in partner_pairs:
        # Generate the column names
        col1 = f'partner_{p1}_trait_{trait}'
        col2 = f'partner_{p2}_trait_{trait}'
        
        # Calculate the difference
        diff_col_name = f'diff_partner_{p1}_{p2}_trait_{trait}'
        diff_df[diff_col_name] = data[col1] - data[col2]

# Merge the differences with the original data
result_df = pd.concat([data, diff_df], axis=1)

# Merge the almost complete data with the metric_run_merge dataframe
# At this point we should have all of the necessary covariates in one dataframe
full_covs_df = pd.merge(result_df, metric_run_merge_df, on='sub')
full_covs_df = full_covs_df.drop('index', axis=1)

# Rename all of the condition columns for clarity
full_covs_df = full_covs_df.rename(columns=
                                                 {"Subject": "sub",
                                                 "partner_1_trait_0": "C_win",
                                                 "partner_1_trait_1": "C_loss",
                                                 "partner_2_trait_0": "S_win",
                                                 "partner_2_trait_1": "S_loss",
                                                 "partner_3_trait_0": "F_win",
                                                 "partner_3_trait_1": "F_loss",
                                                 "diff_partner_3_2_trait_0": "FminS_win",
                                                 "diff_partner_3_1_trait_0": "FminC_win",
                                                 "diff_partner_3_2_trait_1": "FminS_loss",
                                                 "diff_partner_3_1_trait_1": "FminC_loss"})

# Print the first few rows of the resulting DataFrame
display(full_covs_df)

In [ ]:
# Almost finished here, need to create two tables
# First table will have the sub in it for when we go to make sure our inputs are in the proper order for L3
# Second table will be used for copy and pasting into the fsl gui
# Can also create custom tables for various models as seen in cells above (
# This cell will be me for running the correlation matrix

table_sub = full_covs_df[['age_x','sub',
                      'tsnr', 'fdmean',
                      'C_win','C_loss',
                      'S_win','S_loss',
                      'F_win','F_loss',
                      'FminS_win','FminS_loss',
                      'FminC_win', 'FminC_loss']]

table = full_covs_df[['age_x',
                      'tsnr', 'fdmean',
                      'C_win','C_loss',
                      'S_win','S_loss',
                      'F_win','F_loss',
                      'FminS_win','FminS_loss',
                      'FminC_win', 'FminC_loss']]

display(table)

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 8))

# Generate the heatmap
heatmap = sns.heatmap(table.corr(), cmap='vlag', annot=True, fmt=".2f", linewidths=.5)

# Display the heatmap
plt.show()